<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/3.0.0_beta1/OpenHosta/tree/3.0.0_beta1/docs/SmokeTest_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal OpenHosta

In [ ]:
!pip install --upgrade -qqq uv

# If you need to test a pre release uncomment:
VERSION="@3.0.0_beta1"

!uv pip install -U \
    "git+https://github.com/hand-e-fr/OpenHosta.git$VERSION"

In [ ]:
import OpenHosta; print(OpenHosta.__version__)

# Install Ollama

This is to run a local model and have zero dependancies to externa API provides.

In [ ]:
# This seems not to be accepted by google colab anymore. 
# Run it in a separtated terminal i refused
#!apt install -y screen
#!curl -fsSL https://ollama.com/install.sh | sh
#!screen -dmS ollama ollama serve

# So we just show how to install Ollama on linux
!curl -fsSL https://ollama.com/install.sh | sh

We test is ollama is available and how fast Qwen3 runs on it.

In [ ]:
!ollama run qwen3:4b hello --verbose  2>&1 | grep -E ":"

# Use OpenHosta

In [ ]:

from OpenHosta import config

# You can replace with your own API (OpenAI chat/completion compatble)
config.DefaultModel.base_url = "http://localhost:11434/v1"
config.DefaultModel.model_name = "qwen3:4b"
config.DefaultModel.api_key = "not used by ollama local api"


**We test that the API is working with a simple call**

`ask()` makes a very simple call to the API without adding any hidden prompt.


In [ ]:
from OpenHosta import ask

ask("Hello World!")

Most OpenHosta functions are avaiale in sync and async flavor.

Async versions are provided by adding `_async` or by importing from  `from OpenHosta.asynchrone`


In [ ]:
from OpenHosta.asynchrone import ask

# Another way to get the same result is:
# from OpenHosta import ask_async as ask

await ask("Hello World!")


# The emulate function

The main purpose of OpenHosta is to let you organize your code in python and let you decide later if you need to implement function bodies in native python or let an LLM do the job. 

This is really pythonic as types and doc strings will remain unchanged if you decide to replace `return emulate()` by your own code.

In [ ]:
from OpenHosta import emulate

def translate(text:str, language:str)->str:
    """
    This function translates the text in the “text” parameter into the language specified in the “language” parameter.
    """
    return emulate()

result = translate("Hello World!", "French")
print(result)

In very simple words, `emulate()` does the inspection of the function it is called from and buld a prompt that delegated computation of the function to an LLM.
This is very usefull for NLP based functions, but also **smart** decisions in your workflow.

In [ ]:
from enum import Enum

class Choice(Enum):
    """
    WebSearch = "Generic Subject, but not enough confidence in LLM knowledge"
    UseLLM    = "Generic Subject, obviouse answer."
    UserRAG   = "Company private subject, need to search for internal documentation through RAG"
    """
    WebSearch = "WebSearch"
    UseLLM    = "UseLLM"
    UserRAG   = "UserRAG"

In [ ]:
from OpenHosta import emulate

def decide_if_search_or_answer(query: str) -> Choice:
    """
    Decide to search for answer elements in one of the multiple Choice locations.
    
    When the LLM is usure about the correct answer it shall query external data sources.
    
    Args:
        subject (str): The question that we are working on
    Return:
        Choice: the best strategy to use
    """
    return emulate()

In [ ]:
decide_if_search_or_answer("april 2021 is after mars 2020?")


In [ ]:
decide_if_search_or_answer("who won the last NFL cup?")


In [ ]:
decide_if_search_or_answer("when was the first soccer world cup?")


# Inspect LLM Dialog

Ok, but can I see the prompt behind and what was really sent to the API?
-YES-

And the thinking and answer of the LLM ?
-YES-

In [ ]:
from OpenHosta import print_last_prompt

print_last_prompt(decide_if_search_or_answer)

You can also see and modify the prompt before it is sent to the LLM.

In [ ]:
from OpenHosta.core.meta_prompt import EMULATE_META_PROMPT, USER_CALL_META_PROMPT
from OpenHosta import MetaPrompt
 
config.DefaultPipeline.emulate_meta_prompt = MetaPrompt(EMULATE_META_PROMPT.source)
config.DefaultPipeline.user_call_meta_prompt = MetaPrompt(USER_CALL_META_PROMPT.source)


# Go further with agents

There is another notebook that shows how to use agents and memory with OpenHosta.
[Agent with Memory](Agent.ipynb)